In [2]:
import os
import sqlite3
from kivy.app import App
from kivy.uix.boxlayout import BoxLayout
from kivy.uix.textinput import TextInput
from kivy.uix.button import Button
from kivy.uix.gridlayout import GridLayout
from kivy.uix.label import Label
from kivy.uix.scrollview import ScrollView
from kivy.uix.codeinput import CodeInput
from pygments.lexers.sql import SqlLexer

# Deaktiviere das Argument-Parsen von Kivy (wichtig in Jupyter)
os.environ['KIVY_NO_ARGS'] = '1'

# Pfad zur Chinook-Datenbank (bitte anpassen, falls notwendig)
DB_PATH = r"./POC-LangChain/chinook-database-master/ChinookDatabase/DataSources/Chinook_Sqlite.sqlite"

class SQLApp(App):
    # Überschreibe load_kv, damit keine .kv-Datei geladen wird
    def load_kv(self, *args, **kwargs):
        pass

    def build(self):
        self.main_layout = BoxLayout(orientation='vertical', padding=10, spacing=10)
        
        # CodeInput anstelle von TextInput verwenden
        self.input = CodeInput(
            lexer=SqlLexer(),
            size_hint=(1, 0.2),
            multiline=True
        )
        self.main_layout.add_widget(self.input)
        
        # Button zum Ausführen der Abfrage
        execute_btn = Button(text='Abfrage ausführen', size_hint=(1, 0.1))
        execute_btn.bind(on_press=self.execute_query)
        self.main_layout.add_widget(execute_btn)
        
        # ScrollView für die Ergebnisanzeige
        self.scroll_view = ScrollView(size_hint=(1, 0.7))
        # Ein GridLayout, das dynamisch Zeilen (Header und Datenzeilen) aufnimmt
        self.result_grid = GridLayout(cols=1, spacing=5, size_hint_y=None)
        self.result_grid.bind(minimum_height=self.result_grid.setter('height'))
        self.scroll_view.add_widget(self.result_grid)
        self.main_layout.add_widget(self.scroll_view)
        
        return self.main_layout

    def clear_table(self):
        # Entferne alle Widgets aus dem Ergebnisbereich
        self.result_grid.clear_widgets()

    def execute_query(self, instance):
        query = self.input.text.strip()
        if not query:
            self.clear_table()
            self.result_grid.add_widget(Label(text="Bitte gib ein SQL-Statement ein."))
            return

        try:
            # Verbindung zur SQLite-Datenbank herstellen
            conn = sqlite3.connect(DB_PATH)
            cursor = conn.cursor()
            cursor.execute(query)
            results = cursor.fetchall()
            # Spaltennamen abrufen (falls vorhanden)
            columns = [desc[0] for desc in cursor.description] if cursor.description else []
            conn.close()
            
            self.clear_table()
            
            if not results:
                self.result_grid.add_widget(Label(text="Abfrage erfolgreich ausgeführt. Keine Ergebnisse."))
                return

            # Erstelle eine Kopfzeile als GridLayout
            header_layout = GridLayout(cols=len(columns), size_hint_y=None, height=40, padding=5)
            for col in columns:
                header_layout.add_widget(Label(text=str(col), bold=True))
            self.result_grid.add_widget(header_layout)
            
            # Füge die Datenzeilen hinzu
            for row in results:
                row_layout = GridLayout(cols=len(columns), size_hint_y=None, height=30, padding=5)
                for cell in row:
                    row_layout.add_widget(Label(text=str(cell)))
                self.result_grid.add_widget(row_layout)
            
        except sqlite3.Error as e:
            self.clear_table()
            self.result_grid.add_widget(Label(text=f"Fehler bei der Abfrage: {e}"))

# App ausführen
if __name__ == '__main__':
    SQLApp().run()
